In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import gc

In [2]:
# Carga de los datasets en dataframes.
df_amazon = pd.read_csv('Datasets/amazon_prime_titles.csv',delimiter=',')
df_disney = pd.read_csv('Datasets/disney_plus_titles.csv', delimiter=',')
df_hulu = pd.read_csv('Datasets/hulu_titles.csv',delimiter=',')
df_netflix = pd.read_csv('Datasets/netflix_titles.csv',delimiter=',')
df_ratings1 = pd.read_csv('Datasets/ratings/1.csv',delimiter=',')
df_ratings2 = pd.read_csv('Datasets/ratings/2.csv',delimiter=',')
df_ratings3 = pd.read_csv('Datasets/ratings/3.csv',delimiter=',')
df_ratings4 = pd.read_csv('Datasets/ratings/4.csv',delimiter=',')
df_ratings5 = pd.read_csv('Datasets/ratings/5.csv',delimiter=',')
df_ratings6 = pd.read_csv('Datasets/ratings/6.csv',delimiter=',')
df_ratings7 = pd.read_csv('Datasets/ratings/7.csv',delimiter=',')
df_ratings8 = pd.read_csv('Datasets/ratings/8.csv',delimiter=',')

In [3]:
#Creación de un solo dataframe de ratings
df_ratings = pd.concat([df_ratings1,df_ratings2,df_ratings3,df_ratings4,df_ratings5,df_ratings6,df_ratings7,df_ratings8,])
df_ratings.shape

(11024289, 4)

In [4]:
df_ratings.head()

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [5]:
#FUNCION DE LIMPIEZA Y TRANSFORMACION PARA LOS DATASETS
def ETL (df):
    #Extracción del nombre del data set cargado
    nombre_df = [nombre for nombre, valor in globals().items() if valor is df][0]
    print ("Proceso de ETL para el dataset:",nombre_df)
        
    #generar y completar el campo id con las claves de show_id
    df['id'] = nombre_df[3] + df['show_id']
    columna_d = df.pop('id')
    df.insert(0, 'id', columna_d)
    print ("-se a agregado la columna id y se ha generado la clave 'id' a partir de los registros del campo 'show_id'")
   
    #comprobar si hay valores nulos en el campo "rating"
    cant_nulos = df["rating"].isnull().sum()
   
    #reemplaza los datos nulos de la columna "rating" por la letra "G"
    if cant_nulos > 0 :
       df['rating'].fillna("G", inplace=True)
    print ("-se han reemplazado ",cant_nulos,"regsitros del campo 'rating' por el valor 'G' general for all audiences " )
    
    #convertir a minusculas todos los datos de tipo str
    # Convertir columnas tipo objeto a string
    df = df.astype(str)
    # Recorrer cada columna y transformar los textos a minúsculas
    for col in df.columns:
        if df[col].dtypes == 'object':
            df[col] = df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)
    print ("-Todos los textos han sido pasados a minusculas")
    
    # Convertir la columna 'data_added' a tipo date con el formato 'AAAA-mm-dd'
    df['date_added'] = df['date_added'].str.lstrip()
    df['date_added'] = pd.to_datetime(df['date_added'], format='%B %d, %Y')
    print ("-Se ha cambiado el tipo de dato y el formato al campo 'date_added'")
    
    # crear dos nuevas columnas cortando los regsitros de la columna duration por el espacio
    df[['duration_int', 'duration_type']] = df['duration'].str.split(' ', expand=True)

    # Convertir columna_entero a tipo entero
    df['duration_int'] = pd.to_numeric(df['duration_int'], errors='coerce')

    # Dejar como season todas las unidades correspondientes a temporada en el campo duration_type
    df['duration_type'] = df['duration_type'].replace('seasons', 'season')

    print ("-Se han generado las columnas 'duration_int' y 'duration_type' con la informacion del campo duration")
    
    # eliminaR las columnas que se transformaron con nuevos datos.
    df = df.drop(['show_id', 'duration'], axis=1)
    print ("-Se han eliminado las columnas 'show_id', 'duration'")
    
   # Exportar el datframe limpiado y transformado como csv en una ruta especifica.
    ruta_exportacion = "../CSV ETL/" + nombre_df + ".csv"
    df.to_csv(ruta_exportacion, index=False)  
    print(f"-El DataFrame se ha exportado exitosamente como {nombre_df}. en la carpeta /CSV ETL ")

    return df

In [6]:
df_amazon=ETL (df_amazon)
df_amazon.head(3)

Proceso de ETL para el dataset: df_amazon
-se a agregado la columna id y se ha generado la clave 'id' a partir de los registros del campo 'show_id'
-se han reemplazado  337 regsitros del campo 'rating' por el valor 'G' general for all audiences 
-Todos los textos han sido pasados a minusculas
-Se ha cambiado el tipo de dato y el formato al campo 'date_added'
-Se han generado las columnas 'duration_int' y 'duration_type' con la informacion del campo duration
-Se han eliminado las columnas 'show_id', 'duration'
-El DataFrame se ha exportado exitosamente como df_amazon. en la carpeta /CSV ETL 


,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,113,min
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,74,min


In [7]:
df_disney=ETL (df_disney)
df_disney.head(3)

Proceso de ETL para el dataset: df_disney
-se a agregado la columna id y se ha generado la clave 'id' a partir de los registros del campo 'show_id'
-se han reemplazado  3 regsitros del campo 'rating' por el valor 'G' general for all audiences 
-Todos los textos han sido pasados a minusculas
-Se ha cambiado el tipo de dato y el formato al campo 'date_added'
-Se han generado las columnas 'duration_int' y 'duration_type' con la informacion del campo duration
-Se han eliminado las columnas 'show_id', 'duration'
-El DataFrame se ha exportado exitosamente como df_disney. en la carpeta /CSV ETL 


,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,ds1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",nan,2021-11-26,2016,tv-g,"animation, family",join mickey and the gang as they duck the halls!,23,min
1,ds2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",nan,2021-11-26,1988,pg,comedy,santa claus passes his magic bag to a new st. ...,91,min
2,ds3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,"animation, comedy, family",sid the sloth is on santa's naughty list.,23,min


In [8]:
df_hulu=ETL (df_hulu)
df_hulu.head(3)

Proceso de ETL para el dataset: df_hulu
-se a agregado la columna id y se ha generado la clave 'id' a partir de los registros del campo 'show_id'
-se han reemplazado  520 regsitros del campo 'rating' por el valor 'G' general for all audiences 
-Todos los textos han sido pasados a minusculas
-Se ha cambiado el tipo de dato y el formato al campo 'date_added'
-Se han generado las columnas 'duration_int' y 'duration_type' con la informacion del campo duration
-Se han eliminado las columnas 'show_id', 'duration'
-El DataFrame se ha exportado exitosamente como df_hulu. en la carpeta /CSV ETL 


,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,hs1,movie,ricky velez: here's everything,nan,nan,nan,2021-10-24,2021,tv-ma,"comedy, stand up",​comedian ricky velez bares it all with his ho...,NaN,None
1,hs2,movie,silent night,nan,nan,nan,2021-10-23,2020,g,"crime, drama, thriller","mark, a low end south london hitman recently r...",94.0,min
2,hs3,movie,the marksman,nan,nan,nan,2021-10-23,2021,pg-13,"action, thriller",a hardened arizona rancher tries to protect an...,108.0,min


In [9]:
df_netflix=ETL (df_netflix)
df_netflix.head(3)

Proceso de ETL para el dataset: df_netflix
-se a agregado la columna id y se ha generado la clave 'id' a partir de los registros del campo 'show_id'
-se han reemplazado  4 regsitros del campo 'rating' por el valor 'G' general for all audiences 
-Todos los textos han sido pasados a minusculas
-Se ha cambiado el tipo de dato y el formato al campo 'date_added'
-Se han generado las columnas 'duration_int' y 'duration_type' con la informacion del campo duration
-Se han eliminado las columnas 'show_id', 'duration'
-El DataFrame se ha exportado exitosamente como df_netflix. en la carpeta /CSV ETL 


,id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type
0,ns1,movie,dick johnson is dead,kirsten johnson,nan,united states,2021-09-25,2020,pg-13,documentaries,"as her father nears the end of his life, filmm...",90.0,min
1,ns2,tv show,blood & water,nan,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",2.0,season
2,ns3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",nan,2021-09-24,2021,tv-ma,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,1.0,season


In [10]:
#CREACION DE UN SOLO DATA FRAME CON TODAS LAS PLATAFORMAS.
df_streaming = pd.concat([df_amazon,df_disney,df_hulu,df_netflix])
df_streaming.to_csv("../CSV ETL/df_streaming.csv", index=False)


In [11]:
#ETL para el df_ratings

#cambiar el nombre de la columna movieId por id
df_ratings = df_ratings.rename(columns={'movieId': 'id'})

#cambiar el nombre de la columna ratings por score
df_ratings = df_ratings.rename(columns={'rating': 'score'})

#verificar cantidad de datos nulos de la columna score
print (df_ratings['score'].isnull().sum())

#Eliminar columnas irrelevantes.
df_ratings = df_ratings.drop(['timestamp'], axis=1)

df_ratings.shape

0


(11024289, 3)

In [12]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   userId  int64  
 1   score   float64
 2   id      object 
dtypes: float64(1), int64(1), object(1)
memory usage: 336.4+ MB


In [13]:
df_ratings.head()

,userId,score,id
0,1,1.0,as680
1,1,4.5,ns2186
2,1,5.0,hs2381
3,1,5.0,ns3663
4,1,5.0,as9500


In [14]:
df_ratings['id'] = df_ratings['id'].astype('category')
df_ratings['userId'] = df_ratings['userId'].astype('int16')
df_ratings['score'] = df_ratings['score'].astype('float32')

In [15]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 3 columns):
 #   Column  Dtype   
---  ------  -----   
 0   userId  int16   
 1   score   float32 
 2   id      category
dtypes: category(1), float32(1), int16(1)
memory usage: 168.9 MB


In [16]:
df_ratings.memory_usage(deep=True)

Index     88194312
userId    22048578
score     44097156
id        24021448
dtype: int64

In [17]:
df_ratings.to_csv("../CSV ETL/df_ratings.csv", index=False)

In [18]:
# Data frame de promedios de rating

#Eliminar columnas irrelevantes.
df_ratings = df_ratings.drop(['userId'], axis=1)

#agrupar el df por id y obtener los pormedios del score por id 
promedio_score = df_ratings.groupby('id')['score'].mean().round(2)
df_score = pd.DataFrame({'id': promedio_score.index, 'score': promedio_score.values})

#exportar el df_scores como archivo csv.
df_score.to_csv("../CSV ETL/df_score.csv", index=False,)
df_score.head()


,id,score
0,as1,3.47
1,as10,3.44
2,as100,3.61
3,as1000,3.56
4,as1001,3.59
